In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.layers import GlobalAveragePooling2D, BatchNormalization
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv2D
import keras
from keras.callbacks import ModelCheckpoint
from keras.applications.mobilenet import MobileNet  
from sklearn.model_selection import train_test_split
from google.colab.patches import cv2_imshow
import os
import pickle
import cv2
import numpy as np
import pandas as pd
import glob

In [0]:
def collect_data(dirname, csvname):
    with open(filepath, 'ab') as f:
      data = []
      csv = pd.read_csv(csvname, sep=',')
      joy_values = csv['wheel'].values.tolist()
      images = glob.glob(dirname)
      count = 0
      for img in images:
          screenshot = cv2.imread(img)
          if count < len(joy_values):
              screenshot = transform_image(np.array(screenshot))
              data.append([screenshot, joy_values[count]])
          if count == len(images) - 1:
              print('Collected data count - {0}.'.format(count))
              pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)
              data = [] 
          count += 1
def read_data(split=False):
    with open(filepath, 'rb') as f:
        data = []
        while True:
            try:
                temp = pickle.load(f)

                if type(temp) is not list:
                    temp = np.ndarray.tolist(temp)

                data = data + temp
            except EOFError:
                break
        if split:
            x_train = []
            y_train = []

            for i in range(0, len(data)):
                x_train.append(data[i][0])
                y_train.append(data[i][1])

            return np.array(x_train), np.array(y_train)
        else:
            return np.array(data)
def create_model():
    nrows = 66
    ncols = 200
    img_channels = 3
    output_size = 1

    base_mobilenet_model = MobileNet(input_shape =  (66,200,3), 
                                    include_top = False, 
                                    weights = None)
    pilot_model = Sequential()
    pilot_model.add(BatchNormalization(input_shape =(66,200,3)))
    pilot_model.add(base_mobilenet_model)
    pilot_model.add(BatchNormalization())
    pilot_model.add(GlobalAveragePooling2D())
    pilot_model.add(Dropout(0.5))
    pilot_model.add(Dense(1, activation = 'linear' )) # linear is what 16bit did

    pilot_model.summary()

    pilot_model.compile(loss=keras.losses.mean_squared_error,
                  optimizer=keras.optimizers.Adam(lr=0.001),
                  metrics=[sign_pred])
    return pilot_model

def get_model():
    if os.path.isfile(modelpath):
        model = keras.models.load_model(modelpath)
    else:
        model = create_model()
    return model

def sign_pred(y_true, y_pred):
    mult = y_true * y_pred
    return keras.backend.mean(keras.backend.equal(keras.backend.sign(mult),keras.backend.ones_like(mult)), axis = -1)

def train_model(model):
    x, y = read_data(True)

    # test data set 0.15 : training data set 85%
    x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.15, random_state=1)

    checkpoint = ModelCheckpoint('model-{epoch:03d}.h5', monitor='val_loss', verbose=1, mode='min')
    earlystop = EarlyStopping(monitor='sign_pred', mode='max', verbose=1, patience=50)
    model.fit(x_train, y_train, epochs=200, batch_size=64, verbose=1,
              validation_data=(x_valid, y_valid), callbacks=[checkpoint,earlystop])

    model.save(modelpath)
def display_data():
    data = read_data()
    data_size = len(data)
    print('Data size  -  ' + str(data_size))

    for i in range(data_size - 1, 0, -1):
        print(str(image[1]))
        cv2.waitKey(50)
    



In [0]:
filepath = '/content/drive/My Drive/before/sample.dat'
# collect_data('/content/drive/My Drive/before/20200505191234/*.jpg', '/content/drive/My Drive/before/20200505191234/20200505191234.csv')
# collect_data('/content/drive/My Drive/before/20200505191324/*.jpg', '/content/drive/My Drive/before/20200505191324/20200505191324.csv')
# collect_data('/content/drive/My Drive/before/20200505191424/*.jpg', '/content/drive/My Drive/before/20200505191424/20200505191424.csv')
# collect_data('/content/drive/My Drive/before/20200505191517/*.jpg', '/content/drive/My Drive/before/20200505191517/20200505191517.csv')
# collect_data('/content/drive/My Drive/before/20200505191837/*.jpg', '/content/drive/My Drive/before/20200505191837/20200505191837.csv')
# collect_data('/content/drive/My Drive/before/20200505191944/*.jpg', '/content/drive/My Drive/before/20200505191944/20200505191944.csv')
# collect_data('/content/drive/My Drive/before/20200505192054/*.jpg', '/content/drive/My Drive/before/20200505192054/20200505192054.csv')


In [13]:
modelpath = '/content/drive/My Drive/before/mymodel1.h5'
model = get_model()
train_model(model)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_3 (Batch (None, 66, 200, 3)        12        
_________________________________________________________________
mobilenet_1.00_66 (Model)    (None, 2, 6, 1024)        3228864   
_________________________________________________________________
batch_normalization_4 (Batch (None, 2, 6, 1024)        4096      
_________________________________________________________________
global_average_pooling2d_2 ( (None, 1024)              0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 1025      
Total params: 3,233,997
Trainable params: 3,210,055
Non-trainable params: 23,942
_______________________________________

In [0]:
model.predict(np.expand_dims(transform_image(cv2.imread('/content/drive/My Drive/before/20200505191234/20200505191234277788.jpg')), axis=0))

array([[0.09332664]], dtype=float32)

ls: cannot access '/content/drive/MyDrive': No such file or directory
